# 🧩 带导航 & 重用模板的 AT 文档生成 Notebook

In [12]:
!pip install pandas jinja2 sphinx sphinx-idf-theme

In [13]:
import os
import json
import re
import pandas as pd
from jinja2 import Environment, FileSystemLoader, Template
from collections import defaultdict

In [14]:
# 初始化 docs/source 结构
DOC_SRC = "docs/source"
DOC_BUILD = "docs/build/html"
os.makedirs(DOC_SRC, exist_ok=True)
os.makedirs(os.path.join(DOC_SRC, "_static"), exist_ok=True)
os.makedirs(os.path.join(DOC_SRC, "_templates"), exist_ok=True)
os.makedirs(os.path.join(DOC_SRC, "2"), exist_ok=True)  # 示例章节

In [15]:
# 写入 conf.py，含主题必要注册配置
conf_path = os.path.join(DOC_SRC, 'conf.py')
with open(conf_path, 'w', encoding='utf-8') as f:
    f.write("""
import os
import sys
sys.path.insert(0, os.path.abspath('.'))

project = 'AT Command Manual'
author = 'Your Name'
release = '1.0'

extensions = ['sphinx_idf_theme']

templates_path = ['_templates']
exclude_patterns = []

html_theme = 'sphinx_idf_theme'
html_static_path = ['_static']
html_css_files = ['custom.css']

pdf_file = None
pdf_filename = None
pdf_url = None
idf_target_title_dict = {}
languages = ['zh_CN']

def setup(app):
    app.add_config_value('pdf_file', None, 'env')
    app.add_config_value('pdf_filename', None, 'env')
    app.add_config_value('pdf_url', None, 'env')
    app.add_config_value('idf_target_title_dict', {}, 'env')
#   app.add_config_value('languages', ['zh_CN'], 'env')
    app.add_css_file('custom.css')
""")
print('✅ conf.py 写入')

✅ conf.py 写入


In [16]:
# 写入 custom.css
css_path = os.path.join(DOC_SRC, '_static', 'custom.css')
with open(css_path, 'w', encoding='utf-8') as f:
    f.write("""
.document { max-width: 800px; margin: auto; padding: 1em; }
.literal-block pre { background-color: #f8f8f8; border: 1px solid #ddd; padding: 0.8em; font-family: Consolas, monospace; line-height: 1.4; white-space: pre-wrap; overflow-x: auto; }
h1, h2, h3 { font-family: 'Segoe UI', Tahoma, sans-serif; color: #333; margin-top: 1em; margin-bottom: 0.6em; }
ul, ol { margin-top: 0.5em; margin-bottom: 0.8em; padding-left: 1.2em; }
a { color: #0066cc; }
a:hover { text-decoration: underline; }
.literal-block { margin-top: 0.5em; margin-bottom: 1em; }
""")
print('✅ custom.css 写入')

✅ custom.css 写入


In [17]:
# 写入 layout.html override 模板（带导航底部链接）
layout_path = os.path.join(DOC_SRC, '_templates', 'layout.html')
with open(layout_path, 'w', encoding='utf-8') as f:
    f.write("""
{% extends "!layout.html" %}
{% block header %}
  <div class="doc-header">
    <h1>{{ project }}</h1>
  </div>
  {{ super() }}
{% endblock %}
{% block sidebar1 %}
  {{ super() }}
  <div class="custom-sidebar-footer">
    <hr>
    <p><a href="{{ pathto('index', 1) }}">📄 返回目录</a></p>
  </div>
{% endblock %}
""")
print('✅ layout override 写入')

✅ layout override 写入


In [18]:
# 读取 CSV + 展开子类型 + 构建列表
CSV_PATH = "at_commands_full_template.csv"
OUTPUT_DIR = DOC_SRC

def expand_multitype_row(row):
    types = [t.strip() for t in str(row.get('命令类型', '')).split(';')]
    fmts = [t.strip() for t in str(row.get('命令格式', '')).split(';')]
    raw_corr = str(row.get('响应校正', '')).strip().strip("'''")
    if raw_corr:
        resps = raw_corr.split(';')
    else:
        resps = [t.strip() for t in str(row.get('响应', '')).split(';')]

    example_text = str(row.get('示例命令', '') or '').strip()

    max_len = max(len(types), len(fmts), len(resps), 1)
    expanded = []
    for i in range(max_len):
        expanded.append({
            '章节': row.get('章节'),
            '命令': row.get('命令'),
            '命令标题': row.get('命令标题'),
            '命令类型': types[i] if i < len(types) else "",
            '命令格式': fmts[i] if i < len(fmts) else "",
            '响应': resps[i] if i < len(resps) else "",
            '示例命令': example_text,
            '功能描述': row.get('功能描述'),
            '备注': row.get('备注'),
            '参数json': row.get('参数json'),
            '顺序': f"{row.get('顺序')}.{i+1}"
        })
    return expanded

df0 = pd.read_csv(CSV_PATH, dtype=str).fillna("")
expanded = []
for _, r in df0.iterrows():
    expanded.extend(expand_multitype_row(r))
df = pd.DataFrame(expanded)
df.columns = df.columns.str.strip()


In [19]:
# 渲染命令 RST + 插入导航链接
TEMPLATE_STRING = """
.. _cmd-{{ command_name | lower }}:

{{ command_name }}：{{ command_title }}
{{ '=' * (command_name | length + command_title | length + 1) }}

{{ description or '' }}

{% for st in subtypes %}
{{ st.type }}命令
{{ (st.type + '命令') | length * '^' }}

**命令：**

::

    {{ st.fmt }}

**响应：**

::

{% for line in st.response.splitlines() %}
    {{ line }}
{% endfor %}

{% endfor %}
参数
^^^^
{% if parameters %}
{% for name, data in parameters.items() %}
- **{{ name }}**：
  {% if data.get('__desc__') %}
    {{ data['__desc__'] }}
  {% endif %}
  {% for key, value in data.items() if key != '__desc__' %}
    - {{ key }}：{{ value }}
  {% endfor %}
{% endfor %}
{% else %}
无
{% endif %}

说明
^^^^
{{ note or '无' }}

.. container:: pager

{% if prev_cmd %}:doc:`← {{ prev_cmd }} <{{ chapter }}/{{ prev_cmd }}>`{% endif %}
{% if next_cmd %} | :doc:`{{ next_cmd }} → <{{ chapter }}/{{ next_cmd }}>`{% endif %}

示例命令
^^^^^^^^

::

{% for st in subtypes %}
{% if st.example %}
{% for line in st.example.splitlines() %}
    {{ line }}
{% endfor %}
{% endif %}
{% endfor %}
"""

env = Environment()
template = env.from_string(TEMPLATE_STRING)

chapter_commands = defaultdict(list)
chapter_names = []

for chap, grp in df.groupby("章节"):
    chap_name = str(chap).strip()
    if chap_name not in chapter_names:
        chapter_names.append(chap_name)
    chapter_commands.setdefault(chap_name, [])
    chap_dir = os.path.join(OUTPUT_DIR, chap_name)
    os.makedirs(chap_dir, exist_ok=True)

    cmds = list(grp['命令'].unique())
    # 用于导航：建立 prev/next 映射
    for idx, cmd in enumerate(cmds):
        prev_cmd = cmds[idx-1] if idx > 0 else None
        next_cmd = cmds[idx+1] if idx < len(cmds)-1 else None
        subgrp = grp[grp['命令'] == cmd]
        for _, row in subgrp.iterrows():
            st = {
                'type': row.get('命令类型', ''),
                'fmt': row.get('命令格式', ''),
                'response': row.get('响应', ''),
                'example': row.get('示例命令', '')
            }
            subtypes = [st]

            row0 = grp.iloc[0]
            command_title = str(row0.get('命令标题', '')).strip()
            description = str(row0.get('功能描述', '')).strip()
            note = str(row0.get('备注', '')).strip()
            param_json = row0.get('参数json', '{}')
            try:
                parameters = json.loads(param_json)
            except:
                parameters = {}
            if isinstance(parameters, list):
                pdict = {}
                for p in parameters:
                    name = p.get('name', '').strip()
                    desc = p.get('desc', '').strip()
                    valmap = p.get('valmap', {}) or {}
                    pdict[name] = {'__desc__': desc}
                    for k, v in valmap.items():
                        pdict[name][k] = v
                parameters = pdict

            content = template.render(
                command_name=cmd,
                command_title=command_title,
                description=description,
                note=note,
                parameters=parameters,
                subtypes=subtypes,
                prev_cmd=prev_cmd,
                next_cmd=next_cmd,
                chapter=chap_name
            )
            out_path = os.path.join(chap_dir, f"{cmd}.rst")
            with open(out_path, 'w', encoding='utf-8') as f:
                f.write(content)
    chapter_commands[chap_name].extend(cmds)
print('✅ RST 文档生成完毕')

✅ RST 文档生成完毕


In [20]:
# 生成 index.rst
chapter_tmpl = """{{ chapter }}
{{ '=' * (chapter|length) }}

.. toctree::
   :maxdepth: 1

{% for c in cmds %}   {{ c }}
{% endfor %}
"""

main_tmpl = """AT 指令文档
===============

.. toctree::
   :maxdepth: 1
   :caption: 章节目录

{% for ch in chapter_names %}   {{ ch }}/index
{% endfor %}
"""

from jinja2 import Template
for chap, cmds in chapter_commands.items():
    idx = os.path.join(OUTPUT_DIR, chap, 'index.rst')
    with open(idx, 'w', encoding='utf-8') as f:
        f.write(Template(chapter_tmpl).render(chapter=chap, cmds=cmds))

main_idx = os.path.join(OUTPUT_DIR, 'index.rst')
with open(main_idx, 'w', encoding='utf-8') as f:
    f.write(Template(main_tmpl).render(chapter_names=chapter_names))
print('✅ index.rst 完成')

✅ index.rst 完成


In [21]:
!sphinx-build -b html docs/source docs/build/html -c docs/source
print("✅ HTML 构建完成")

Running Sphinx v7.4.7
loading translations [en]... done
loading pickled environment... done
building [mo]: targets for 0 po files that are out of date
writing output... 
building [html]: targets for 3 source files that are out of date
updating environment: [extensions changed ('sphinx_idf_theme')] 3 added, 0 changed, 0 removed
reading sources... [ 33%] 2/AT+CSMS
reading sources... [ 67%] 2/index
reading sources... [100%] index

looking for now-outdated files... none found
pickling environment... done
checking consistency... done
preparing documents... done
copying assets... 
copying static files... Copying the source path c:\Users\txiab\Documents\Git-folder\Building-docs\docs-as-code-learning\learning-code\csv2pdf-tool\.venv\Lib\site-packages\sphinx_idf_theme\static\css/badge_only.css to C:\Users\txiab\Documents\Git-folder\Building-docs\docs-as-code-learning\learning-code\csv2pdf-tool\docs\build\html\_static/css\badge_only.css will overwrite data, as a file already exists at the destin

C:\Users\txiab\Documents\Git-folder\Building-docs\docs-as-code-learning\learning-code\csv2pdf-tool\docs\source\2/AT+CSMS.rst:5: WARNING: Title underline too short.

AT+CSMS：选择短信服务
C:\Users\txiab\Documents\Git-folder\Building-docs\docs-as-code-learning\learning-code\csv2pdf-tool\docs\source\2/AT+CSMS.rst:11: WARNING: Title underline too short.

测试命令
^^^^
C:\Users\txiab\Documents\Git-folder\Building-docs\docs-as-code-learning\learning-code\csv2pdf-tool\docs\source\2/AT+CSMS.rst:60: ERROR: Content block expected for the "container" directive; none found.
